# 🎬 Wan 2.1 Video Studio - Colab Launcher

이 노트북은 Google Colab에서 Wan 2.1 Video Studio를 실행합니다.
T4 GPU 이상이 필요합니다.

**실행 방법**: 각 셀을 순서대로 실행하세요.

In [1]:
# GPU 및 시스템 메모리 확인
import torch, psutil

assert torch.cuda.is_available(), "GPU가 필요합니다. 런타임 → 런타임 유형 변경 → GPU를 선택하세요."
gpu_name = torch.cuda.get_device_name(0)
vram_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
ram_gb = psutil.virtual_memory().total / (1024**3)

print(f"✅ GPU: {gpu_name}")
print(f"✅ VRAM: {vram_gb:.1f} GB")
print(f"✅ System RAM: {ram_gb:.1f} GB")

if vram_gb < 14:
    print("⚠️ VRAM이 14GB 미만입니다. 480P 모드만 안정적으로 사용 가능합니다.")

if ram_gb < 20:
    print("⚠️ 시스템 RAM이 20GB 미만입니다. 14B 모델은 최소 20GB 이상 필요합니다.")
    print("💡 해결: Colab 메뉴 → 런타임 → 런타임 유형 변경 → 'High-RAM' 선택")
    print("   또는 왼쪽 하단에 RAM 부족 알림 시 '더 많은 RAM 확보' 클릭")

✅ GPU: Tesla T4
✅ VRAM: 14.7 GB
✅ System RAM: 12.7 GB
⚠️ 시스템 RAM이 20GB 미만입니다. 14B 모델은 최소 20GB 이상 필요합니다.
💡 해결: Colab 메뉴 → 런타임 → 런타임 유형 변경 → 'High-RAM' 선택
   또는 왼쪽 하단에 RAM 부족 알림 시 '더 많은 RAM 확보' 클릭


In [ ]:
# 앱 코드 설치 + 전체 환경 셋업
import os

APP_DIR = "/content/image-video-studio"

# Clone or update app code
if not os.path.exists(APP_DIR):
    !git clone https://github.com/hyunlord/image-video-studio.git {APP_DIR}
else:
    !git -C {APP_DIR} pull --ff-only
    print("앱 코드가 이미 존재합니다. (최신 pull 완료)")

%cd {APP_DIR}
!pip install -q -r requirements.txt

# .env 파일이 있으면 환경변수 로드
from pathlib import Path
env_file = Path(APP_DIR) / ".env"
if env_file.exists():
    for line in env_file.read_text().splitlines():
        line = line.strip()
        if line and not line.startswith("#") and "=" in line:
            key, _, value = line.partition("=")
            if value.strip():
                os.environ[key.strip()] = value.strip()
    print("✅ .env 환경변수 로드 완료")
else:
    print("💡 .env 파일 없음 (선택사항: .env.sample 참고하여 생성)")

In [ ]:
# Run common setup script (Wan 2.1, model download, post-processing tools)
!bash {APP_DIR}/scripts/setup.sh --base-dir /content

print("\n✅ 전체 환경 셋업 완료")

In [ ]:
# 기존 프로세스 정리 + 서버 시작 + 터널링
import os, sys, threading, subprocess

# Kill any existing ngrok/server processes
subprocess.run(["pkill", "-f", "ngrok"], capture_output=True)
subprocess.run(["fuser", "-k", "8000/tcp"], capture_output=True)

# Set environment variables
os.environ["WAN21_DIR"] = "/content/Wan2.1"
os.environ["CODEFORMER_DIR"] = "/content/CodeFormer"
os.environ["RIFE_DIR"] = "/content/RIFE"
os.environ["MODEL_CACHE_DIR"] = "/content/Wan2.1/ckpts/FLF2V-14B-720P"

# Add app to path
APP_DIR = "/content/image-video-studio"
if APP_DIR not in sys.path:
    sys.path.insert(0, APP_DIR)

# Start ngrok tunnel
try:
    from pyngrok import ngrok
except ImportError:
    !pip install -q pyngrok
    from pyngrok import ngrok

ngrok_token = os.environ.get("NGROK_AUTH_TOKEN")
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)

tunnel = ngrok.connect(8000)
public_url = tunnel.public_url
print(f"🌐 Public URL: {public_url}")

# Start uvicorn in background thread
import uvicorn
from backend.app import app

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print(f"\n{'='*50}")
print(f"🎬 Wan 2.1 Video Studio 실행 중!")
print(f"🌐 접속 URL: {public_url}")
print(f"📊 GPU 모니터: {public_url}/monitor")
print(f"{'='*50}")
print(f"\n⚠️ 이 셀을 중단하면 서버가 종료됩니다.")
print(f"💡 위 URL을 클릭하여 웹 브라우저에서 접속하세요.")